Group-By & RFM Analysis
Recency: Did they make a purchase recently?
Frequency: How often do they make a purchase?
Monetary: How much do they spend?

In [1]:
import polars as pl
import pandas as pd
import numpy as np
import pyarrow

import matplotlib.pyplot as plt
import seaborn as sns


# Download Dataset or run in a kaggle notebook
# https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
df = pl.read_csv("C:/Users/LENOVO/Downloads/2019-Oct.csv")

In [2]:
df.shape

(42448764, 9)

In [3]:
df.head().to_pandas()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,None,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,None,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [6]:
# working with dates docs: 
# https://pola-rs.github.io/polars-book/user-guide/howcani/timeseries/parsing_dates_times.html

df = df.with_columns(
    pl.col("event_time").str.strptime(pl.Datetime,"%Y-%m-%d %H:%M:%S %Z")
)

In [7]:
# Useful step to understand the data. Look at one user-id
df.filter(pl.col('user_id') == 520088904).to_pandas().sample(5)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
12,2019-10-31 23:59:23,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
8,2019-10-31 00:55:44,view,1005193,2053013555631882655,electronics.smartphone,meizu,241.71,520088904,c4a5c1d5-0a18-4a59-9f6d-db852139e56c
2,2019-10-28 18:34:47,view,40500455,2090228401527849663,None,powerplant,6.69,520088904,5e511016-454f-4af9-b4b0-f8d0d0618cd2
6,2019-10-31 00:50:07,view,1003416,2053013555631882655,electronics.smartphone,meizu,408.74,520088904,c4a5c1d5-0a18-4a59-9f6d-db852139e56c
14,2019-10-31 23:59:56,view,1003936,2053013555631882655,electronics.smartphone,xiaomi,437.33,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33


1. Keep rows with purchases only

In [8]:
df_purchases = df.filter(pl.col("event_type") == 'purchase')

In [9]:
df_purchases.shape

(742849, 9)

2. Let's compute RFM

In [10]:
# select unique values
df_purchases = df_purchases.select(['event_time', 'user_id', 'price']).unique()

In [11]:
df_purchases.shape

(742773, 3)

In [12]:
df_purchases.head()

event_time,user_id,price
datetime[μs],i64,f64
2019-10-01 02:19:12,516178643,391.26
2019-10-01 02:20:11,517129864,189.91
2019-10-01 02:20:28,554101070,91.12
2019-10-01 02:21:07,516815266,62.52
2019-10-01 02:21:59,514127317,3.37


Compute time difference

In [13]:
from datetime import datetime

anchor_date = datetime(2019, 11, 30)

In [14]:
df_purchases = df_purchases.with_columns(
    (anchor_date - pl.col("event_time")).alias("date_diff") / (1e6 * 3600 * 24)
)

In [15]:
df_purchases.head()

event_time,user_id,price,date_diff
datetime[μs],i64,f64,f64
2019-10-01 02:19:12,516178643,391.26,59.903333
2019-10-01 02:20:11,517129864,189.91,59.90265
2019-10-01 02:20:28,554101070,91.12,59.902454
2019-10-01 02:21:07,516815266,62.52,59.902002
2019-10-01 02:21:59,514127317,3.37,59.9014


In [22]:
#%%timeit
df_agg = df_purchases.group_by('user_id').agg([
    pl.col("date_diff").min().alias("recency"),
    pl.count().alias("frequency"),
    pl.col("price").sum().alias("monetary")
])

In [23]:
df_agg.shape

(347118, 4)

In [24]:
df_agg.head()

user_id,recency,frequency,monetary
i64,f64,u32,f64
519038256,40.822627,2,200.5
556611080,40.734468,2,263.06
512785912,38.319931,2,198.95
518612968,56.929606,1,42.47
515185496,40.761354,1,53.03


3. Convert to pandas for analysis and plotting

In [25]:
df_agg = df_agg.to_pandas()

In [26]:
breaks = np.arange(0, 1.1, 0.1)
num_vars = ['recency', 'frequency', 'monetary']
df_agg[num_vars].quantile(breaks)

,recency,frequency,monetary
0.0,29.000509,1.0,0.880
0.1,31.583488,1.0,42.990
0.2,34.398789,1.0,82.574
0.3,37.014054,1.0,131.020
0.4,39.733875,1.0,174.310
0.5,42.679005,1.0,246.520
0.6,45.592382,1.0,331.972
0.7,48.523569,2.0,483.900
0.8,51.798266,2.0,766.760
0.9,55.703633,4.0,1418.050


In [27]:
num_vars = ['recency', 'frequency', 'monetary']
df_agg[num_vars].quantile([0, 0.25, 0.5, 0.75, 1])

,recency,frequency,monetary
0.00,29.000509,1.0,0.88
0.25,35.637179,1.0,107.59
0.50,42.679005,1.0,246.52
0.75,49.889444,2.0,594.84
1.00,59.995810,321.0,265569.52
